# Simulation scenarios

* Simulation basics
* Verification: Waveforms and assertions
* Writing portable simulations


## Simulation basics

Once a design is conceived, its correct functionality in conjunction with a known good template or routine has to be verified. This is typically done using a (virtual) test bench that tests the unit under test ('UUT') against external stimuli.

A typical test bench has one main stimulus routine defining a *sequence* of signal events.

Since cyrite does not provide a built-in simulator, it normally creates HDL output which is in turn fed to an external simulator such as GHDL or ICARUS. Again, HDL code is generated for the stimulus sequence from the transpiled intermediate representation. The same cyHDL notation however would allow to execute the code in a native Python simulator (like MyHDL).

There are several ways to instance a simulation, however they are not always portable betwee different back ends. Let's start with a simple synchronous unit first, that is translated to HDL for simulation.

In [1]:
from cyhdl import *

Bool = Signal.Type(bool)

@block
def unit(clk : ClkSignal, en: Bool, a: Signal, q : Signal.Output):
    @always(clk.posedge)
    def worker():
        if en:
            q.next = ~a
    
    return worker

### Timing aspects

The above `@block` is a hardware synthesizeable unit, it contains no particular timing specifiation and is clock sensitive.

Once we enter the simulation world of stimuli, time steps come into play. The external clock stimulus for instance is modelled using a timing delay with a given time specification.

This time specification is given in an abstract integer in cycles or time units. The time unit again ist a testbench property and is  normally configured early, by default it is one nanosecond.

It is important to note that not all simulator back ends are compatible to each other in their timing behaviour. The most common simulation strategy is to use `int` values for delays, only.

However it is possible with HDL back ends to use `float`, for example to create a PLL simulation.

### Simple block test bench

A `@block` containing several stimuli can function as a test bench.
Once no more stimuli are present, the simulation is typically halted.

The main stimulus in synchronous design is always its master clock which is here driven by the `clkgen()` function. More complicated clock behaviour such as PLLs show it require a little different approach or inclusion of a black box module. For a waveform output as below, we **always** need a local clock signal for the sampling.

Because all event driven processes or sequences are translated to IRL generator notation internally, you can not use 'yield from' statements in the `@sequence` main stimulus. However, you can make use of `@hdlmacro`s for language-based simulators (explained further below).

In [2]:
@block
def tb_unit(clkname):
    clk = ClkSignal(name=clkname)
    en = Bool()
    a, q = [ Signal(intbv()[8:]) for _ in range(2) ]
    uut = unit(clk, en, a, q)
    
    @always(delay(4))
    def clkgen():
        clk.next = ~clk
        
    @sequence
    def main():
        en.next = False
        yield delay(10)
        yield clk.negedge
        en.next = True
        a.next = 0xaa
        yield clk.negedge
        assert q == 0x55
        yield delay(100)
                
        raise StopSimulation # Terminate without error
        
    
    return instances()

This test bench instances the above `unit`, instances a clock generator using a delay element and runs a `@sequence` of stimuli and checks. Note that by default, the ClkSignal is always initialized to `False`.

To run this test bench on a VHDL simulator for instance, we cast it and pass it to the Simulator API:

In [3]:
clkname = 'clk'
tb = tb_unit(clkname)

s = Simulator(targets.VHDL)
s.run(tb, 80, wavetrace = "tb.vcd")

 Writing 'unit' to file /tmp/myirl_tb_unit_hgndnxvw/unit.vhdl 
 Writing 'tb_unit' to file /tmp/myirl_tb_unit_hgndnxvw/tb_unit.vhdl 
 Creating library file module_defs.vhdl 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg clkname (<class 'str'>)
  base.warnings.warn(msg)


0

When using GHDL, a VCD file with name given by the `wavetrace` parameter is created. To display this file in the notebook, we have to manually import a few wave drawing modules.

The wave utility requires a sample clock, whose clock name must be specified. In this case, it has to match the master clock's name.

In [4]:
from cyrite import waveutils
waveutils.draw_wavetrace(tb, 'tb.vcd', clkname)

## Specific test bench

A test bench can also be specific to a target and not be used with other simulators.
In this case, an extra decorator is prepended to the `@block` function, which specifies the simulator to use. The simulator has a default_target property which is used as elaboration target for the intermediate output.

In [5]:
from cyrite.simulation import sim, icarus, ghdl

@sim.testbench(icarus.ICARUS, 'ns')
@block
def tb_unit2(clkname : PassThrough(str)):
    clk = ClkSignal(name=clkname)
    en = Bool()
    a, q = [ Signal(intbv()[8:]) for _ in range(2) ]
    
    uut = unit(clk, en, a, q)
    
    @always(delay(4))
    def clkgen():
        clk.next = ~clk
        
    @sequence
    def main():
        en.next = False
        yield delay(10)
        yield clk.negedge
        en.next = True
        a.next = 0xaa
        yield clk.negedge
        assert q == 0x55
        yield delay(100)
        
        raise StopSimulation # Terminate without error
        
    
    return instances()


As this is a specific test bench class bound to a simulator, it is the test bench instance that is '.run':

In [6]:
tb = tb_unit2('clk1')
tb.run(200)

 Module tb_unit2: Existing instance unit, rename to unit_1 
 Writing 'unit_1' to file /tmp/myirl_tb_unit2_xuk8h9ud/unit_1.v 
 Writing 'tb_unit2' to file /tmp/myirl_tb_unit2_xuk8h9ud/tb_unit2.v 
 Creating library file module_defs.v 


0

When Co-simulation is used, a test bench actually does **not** translate into IRL, as no HDL is created. In this case, a `@sim.testbench` decorator can occur without a `@block` notation.

Then, the python code is actually executed 'natively'. This is however advanced practise. Examples are found in the CXXRTL Co-Simulation tests. The recommended way is to create a portable test bench as shown below.

## Portable test bench

If a test bench should be run with several different simulator back ends, either for HDL targets or with Co-Simulation, the following derivation from a `cyrite_factory.Module` helps to create output for various architectures.

Here, the cosimulation capable generators are decorated by `@self.always` instead of `@always` and so forth. The reason is that these are depending on the target architecture:
*   HDL-Simulation output: All is transpiled to the target HDL
*   Co-Simulation: Hardware entities are transpiled, testbench processes are executed natively.

**Note**:
* A CXXRTL co-simulation can only have *one* unit under test (UUT)
* The CXXRTL cosimulation interface is currently restricted to signals of maximum 32 bit size. OverflowErrors will occur when passing a large signal through the top interface of a compiled simulation UUT.
* The UUT must be synthesizeable hardware (no `@self.always*` constructs allowed)
* Timing specifications must be integers

In [7]:
class TestDesign(cyrite_factory.Module):
    
    def __init__(self, name, arch, clktoggle_period):
        super().__init__(name, arch)
        self.clkperiod_half = clktoggle_period
    
    @cyrite_factory.testbench('ns')
    def tb_unitx(self):
        clk = self.ClkSignal(name='clk')
        en = self.Signal(bool())
        a, q = [ self.Signal(intbv()[8:]) for _ in range(2) ]

        uut = unit(clk, en, a, q)

        @self.always(delay(self.clkperiod_half))
        def clkgen():
            clk.next = ~clk

        @self.sequence
        def main():
            en.next = False
            yield delay(10)
            yield clk.negedge
            en.next = True
            a.next = 0xaa
            yield clk.negedge
            assert q == 0x55
            yield delay(100)

            raise StopSimulation # Terminate without error


        return instances()        

This test bench is portable among all three simulator architectures below. However, there are differences in the output, as you can see from the wave trace. The simulation will pass for all, though.

In [8]:
from yosys.simulator import CXXRTL

SIMULATOR = icarus.ICARUS
# SIMULATOR = ghdl.GHDL
# SIMULATOR = CXXRTL

design = TestDesign('test', SIMULATOR,
                    clktoggle_period = 3)
tb = design.tb_unitx()

tb.run(200, debug = False, wavetrace = True)


 Declare obj 'tb_unitx' in context '(TestDesign 'test')'(<class '__main__.TestDesign'>) 
 Module test: Existing instance unit, rename to unit_2 
 Writing 'unit_2' to file /tmp/myirl_test__l1enpfy/unit_2.v 
 Writing 'tb_unitx' to file /tmp/myirl_test__l1enpfy/tb_unitx.v 
 Creating library file module_defs.v 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/cyrite/simulation/icarus.py:51: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")


0

In [9]:
waveutils.draw_wavetrace(tb, 'tb_unitx.vcd', 'clk')

### Restrictions and pitfalls

Because `@self.sequence` is run in the co-simulation context when using CXXRTL, some hardware generator specific constructs such as `@hdlmacro` can not just be called like in their true hardware counterparts. Make sure to `.evaluate()` it!

On the other hand, sequential minded macros can not be instanced in hardware, if they contain delays. Therefore, `@*.sequence` type constructs are meant to be called from a simulation context, only.

See [Ports (signal classes)](ports.ipynb) for more details.

## Delay modelling

To model delays in IRL, an explicit `simulation.Delayed` unary operator exists, which delays the specified signal by a given time, either in `int` or `float` in nanoseconds, or a signal of wire type `Time`. The latter are static delays, if a delay depending on a signal (not variable) is desired, for instance, when a VCO model is needed, care must be taken to initialize such a period signal correctly.

**Note**: Because this is a HDL specific simulation construct, you must use the default `Signal` types. Using CoSimulation signals will produce an error. This construct is thus not portable among all simulation contexts

In [10]:
from cyhdl import *

import cyrite.simulation as sim

@block
def dyn_clk(d : float):
    ts = Signal(sim.Time(d)) # in ns
    ts.init = True
    
    s = Signal(bool())
    s.init = True
    
    @always(s, ts)
    def dynclkgen():
        "A dynamic clock generator"
        s.next = sim.VariableDelay(~s, ts)
        
    @sequence
    def main():
        yield sim.delay(5)
        ts.next = 2 * d
        yield sim.delay(4)
        
        raise StopSimulation    
    return instances()    

In [11]:
from myirl import targets
from myirl.test.common_test import Simulator
uut = dyn_clk(0.2)
simulator = Simulator(targets.Verilog)
ret = simulator.run(uut, debug = True)

 Writing 'dyn_clk' to file /tmp/dyn_clk.v 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
ICARUS FILES ['/tmp/dyn_clk.v']
==== COSIM stdout ====
VCD info: dumpfile dyn_clk.vcd opened for output.
Stop Simulation



/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg d (<class 'float'>)
  base.warnings.warn(msg)


### Individual signal delay

If a greater number of signals is to be delayed by an individual delay, the `@assign_delayed()@` macro can be used.


In [12]:
from cyrite.simulation.time import assign_delayed

a, b, c = [ Signal(bool()) for _ in range(3) ]

wires = [
    b     @assign_delayed(2)@     a,
    c     @assign_delayed(3)@     a
]

## Simulator back end issues

Simulations may behave differently, depending on the simulator back ends. A simulation written for one back end may not behave the same on another. For example:

* An asserted signal depending on a previous assignment may be valid immediately or after a delta wait period
* Likewise, signals that depend on a synchronous clock may not be valid right after their driving clock event
* Some simulators like CXXRTL or the MyHDL simulator do not deal with undefined/uninitialized values or have size restriction in the Cosimulation signal interface.

For portable simulators, a few thumb rules apply:

* Validate/assert signals on their opposite clock edge they are updated with
* Use context sensitive macros (`@cyrite_method.sequence` or `@cyrite_factory.Module::hdlmacro` (see [Port signals: Macro extensions](ports.ipynb#Macro-extensions) to insert delays or toggle complex sequences

### Co-Simulation signals

When running a co-simulation back end such as CXXRTL, all top level signals connected to the compiled unit under test are handled by the simulator back end. Those left unconnected however are a priori dummy simulation signals at start up. Once a simulator is instanced, its co-simulation layer will pick up those signals from the returned test bench (`instances()`) and assign them to proper co-simulation signals. If signals are omitted from the return, they may need to be assigned to a specific separate back end. If left with a dummy signal stub, co-simulation will throw an error upon access of a unconnected signal.

Since the cosimulation layer is directing the time steps taken, it also takes care of the cosimulation signal changes.

### CXXRTL extras

The CXXRTL back end in particular is driven by a simple CoSimulation layer that does not resolve complex circular asynchronous dependencies, neither is it sensitive to events or signal changes from within the simulated unit.
It also has a restriction to a maxmimum 32 bit wide Cosimulation signal at this moment.

Also, it is a simulator for pure hardware entities that are either clock synchronous or asynchronous without any delay modelling. Therefore, units that drive a clock such a PLL can not be simulated within CXXRTL.

However, a safe assumption is: All elements that are accepted by yosys for synthesis typically translate to CXXRTL.



### Compilation details

When an unit under test is compiled during a `.run` call, a `.so` will result in the current work directory's `runtime/` folder. When the `recompile` argument is not specified, the unit is built without a specific tag. Upon next `.run`. with `recompile = False`, a cold import will be attempted. 
Upon cold import, a simple signature check according to the interface is made. When running a test sequence, it is recommended to set `recompile=True` to force recompilation of a unit with a given uid tag.

In [13]:
design = TestDesign('test', CXXRTL,
                    clktoggle_period = 3)
tb = design.tb_unitx()

tb.run(200, debug = False)

 Module test: Existing instance unit, rename to unit_3 
 Adding module with name `unit_3` 
 FINALIZE implementation `unit_3` of `unit` 
Compiling /tmp/myirl_test_llcmb2t0/unit.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_test_llcmb2t0/unit.pyx
running build_ext
building 'runtime.unit' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_test_llcmb2t0
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=unit -Iruntime -I/tmp/myirl_test_llcmb2t0/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_test_llcmb2t0/unit.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_test_llcmb2t0/unit.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=unit -Iruntime -I/tmp/myirl_test_llcmb2t0/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_test_llcmb2t0/unit_rtl.cpp -o build/temp.linux-x86_64-3.10/tmp/myi

Using '/tmp/myirl_test_llcmb2t0/' for output
STOP SIMULATION @117


With `recompile=False`, the created module will be cold-imported.
Note we must create a new test bench instance.